# Load, Clean, Format

Here in lies the cleaning process for this project.

    The following datasets were used:

    1. UN Energy Statistics Database, Solar Electricity
        -url: "http://data.un.org/Data.aspx?d=EDATA&f=cmID%3aES%3btrID%3a01"
        -format: csv download
    
    2. UN Energy Statistics Database, Wind Electricity
        -url: "http://data.un.org/Data.aspx?d=EDATA&f=cmID%3aEW"
        -format: csv download
    
    3. International Renewable Energy Agency (IRENA) Database, Renewable Public Investment 2022
        -url: "https://www.irena.org/Data/View-data-by-topic/Finance-and-Investment/Renewable-Energy-Finance-Flows"
        -format: csv download
        
### High Level Summary

        - Solar and Wind sets were in the same format and thus cleaned in similar ways
        - Solar, Wind, and Investment sets were trimmed to only include years 2016 and 2020
        - Investment set, while having a somewhat common "Country" column, contained lots of extraneous data
        - Raw investment set has data down to the wind and solar level, but was spotty at this level
        - Decided to use all renewable investment as the aggregation level to create a fuller picture
        - Ultimate goal was to merge all sets together, but this created too many Nulls as seen in final test merge

In [407]:
# set up dependencies
import pandas as pd


## Solar Data

In [408]:
# read solar file and create dataframe
# spaces after delimiter in file so need to skip
solar_df = pd.read_csv("resources/solar.csv",skipinitialspace=True)

solar_df

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Afghanistan,Electricity - total solar production,2021.0,"Kilowatt-hours, million",32.000,1.0
1,Afghanistan,Electricity - total solar production,2020.0,"Kilowatt-hours, million",31.930,NaN
2,Afghanistan,Electricity - total solar production,2019.0,"Kilowatt-hours, million",20.770,NaN
3,Albania,Electricity - total solar production,2021.0,"Kilowatt-hours, million",40.756,NaN
4,Albania,Electricity - total solar production,2020.0,"Kilowatt-hours, million",32.260,NaN
...,...,...,...,...,...,...
2539,Zambia,Electricity - total solar production,2016.0,"Kilowatt-hours, million",0.163,NaN
2540,Zimbabwe,Electricity - total solar production,2020.0,"Kilowatt-hours, million",24.400,NaN
2541,NaN,NaN,NaN,NaN,NaN,NaN
2542,fnSeqID,Footnote,NaN,NaN,NaN,NaN


In [409]:
# remove Unit and Quantity Footnotes columns
solar_df = solar_df.drop(columns=["Quantity Footnotes","Unit"])

In [410]:
# delete last 3 rows (footnote descriptor and empty row)
solar_df = solar_df.dropna(how='any')
# check to see the last 3 rows were dropped and that all other rows are complete
solar_df.count()

Country or Area            2541
Commodity - Transaction    2541
Year                       2541
Quantity                   2541
dtype: int64

In [411]:
# CLEANING (solar)

# change Year column from dtype float to dtype int to get rid of the ".0"
# then change Year column from dtype int to string
solar_df = solar_df.astype({"Year": "int"}, errors='raise')
solar_df = solar_df.astype({"Year": "str"}, errors='raise')

# change Commodity - Transaction column name to "Technology" and field values to "Solar" 
# for easier readability and future merge
solar_df.rename(columns = {'Commodity - Transaction': 'Technology'}, inplace = True)
solar_df["Technology"] = solar_df["Technology"].replace({"Electricity - total solar production": "Solar"})

# rename Quantity column to "Quantity (kWh, million)"
solar_df.rename(columns = {'Quantity': 'Quantity (kWh, million)'}, inplace = True)

# trim down to only rows with Year = 2016, 2017, 2018, 2019, 2020
solar_df = solar_df.loc[(solar_df["Year"] == "2020")|
                                (solar_df["Year"] == "2019")|
                                (solar_df["Year"] == "2018")|
                                (solar_df["Year"] == "2017")|
                                (solar_df["Year"] == "2016")]


solar_df

,Country or Area,Technology,Year,"Quantity (kWh, million)"
1,Afghanistan,Solar,2020,31.930
2,Afghanistan,Solar,2019,20.770
4,Albania,Solar,2020,32.260
5,Albania,Solar,2019,22.196
6,Algeria,Solar,2020,665.220
...,...,...,...,...
2536,Zambia,Solar,2019,117.510
2537,Zambia,Solar,2018,1.320
2538,Zambia,Solar,2017,0.163
2539,Zambia,Solar,2016,0.163


In [412]:
# solar_df with only countries with year = 2016 and 2020 for final analysis

solar_df_trim = solar_df.loc[(solar_df["Year"] == "2020") | (solar_df["Year"] == "2016")]
solar_df_trim

,Country or Area,Technology,Year,"Quantity (kWh, million)"
1,Afghanistan,Solar,2020,31.930
4,Albania,Solar,2020,32.260
6,Algeria,Solar,2020,665.220
10,Algeria,Solar,2016,339.100
14,American Samoa,Solar,2020,5.210
...,...,...,...,...
2524,Yemen,Solar,2020,489.800
2528,Yemen,Solar,2016,155.000
2535,Zambia,Solar,2020,150.156
2539,Zambia,Solar,2016,0.163


## Wind Data

In [413]:
# read wind file and create dataframe
# spaces after delimiter in file so need to skip
wind_df = pd.read_csv("resources/wind.csv",skipinitialspace=True)

wind_df

,Country or Area,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes
0,Algeria,Electricity - total wind production,2020.0,"Kilowatt-hours, million",7.701,NaN
1,Algeria,Electricity - total wind production,2019.0,"Kilowatt-hours, million",11.690,1.0
2,Algeria,Electricity - total wind production,2018.0,"Kilowatt-hours, million",10.520,1.0
3,Algeria,Electricity - total wind production,2017.0,"Kilowatt-hours, million",19.400,1.0
4,Algeria,Electricity - total wind production,2016.0,"Kilowatt-hours, million",19.400,1.0
...,...,...,...,...,...,...
2236,Viet Nam,Electricity - total wind production,2009.0,"Kilowatt-hours, million",10.000,NaN
2237,Viet Nam,Electricity - total wind production,2008.0,"Kilowatt-hours, million",1.000,NaN
2238,NaN,NaN,NaN,NaN,NaN,NaN
2239,fnSeqID,Footnote,NaN,NaN,NaN,NaN


In [414]:
# remove Unit and Quantity Footnotes columns
wind_df = wind_df.drop(columns=["Quantity Footnotes","Unit"])

In [415]:
# delete last 3 rows (footnote descriptor and empty row)
wind_df = wind_df.dropna(how='any')
# check to see the last 3 rows were dropped and that all other rows are complete
wind_df.count()

Country or Area            2238
Commodity - Transaction    2238
Year                       2238
Quantity                   2238
dtype: int64

In [416]:
# CLEANING (wind)

# change Year column from dtype float to dtype int to get rid of the ".0"
# then change Year column from dtype int to string
wind_df = wind_df.astype({"Year": "int"}, errors='raise')
wind_df = wind_df.astype({"Year": "str"}, errors='raise')

# change Commodity - Transaction column name to "Technology" and field values to "Solar" 
# for easier readability and future merge
wind_df.rename(columns = {'Commodity - Transaction': 'Technology'}, inplace = True)
wind_df["Technology"] = wind_df["Technology"].replace({"Electricity - total wind production": "Wind"})

# rename Quantity column to "Quantity (kWh, million)"
wind_df.rename(columns = {'Quantity': 'Quantity (kWh, million)'}, inplace = True)

# trim down to only rows with Year = 2016, 2017, 2018, 2019, 2020
wind_df = wind_df.loc[(wind_df["Year"] == "2020")|
                                (wind_df["Year"] == "2019")|
                                (wind_df["Year"] == "2018")|
                                (wind_df["Year"] == "2017")|
                                (wind_df["Year"] == "2016")]


wind_df

,Country or Area,Technology,Year,"Quantity (kWh, million)"
0,Algeria,Wind,2020,7.701
1,Algeria,Wind,2019,11.690
2,Algeria,Wind,2018,10.520
3,Algeria,Wind,2017,19.400
4,Algeria,Wind,2016,19.400
...,...,...,...,...
2225,Viet Nam,Wind,2020,976.000
2226,Viet Nam,Wind,2019,312.000
2227,Viet Nam,Wind,2018,312.000
2228,Viet Nam,Wind,2017,270.000


In [417]:
# wind_df with only countries with year = 2016 and 2020 for final analysis

wind_df_trim = wind_df.loc[(wind_df["Year"] == "2020") | (wind_df["Year"] == "2016")]
wind_df_trim

,Country or Area,Technology,Year,"Quantity (kWh, million)"
0,Algeria,Wind,2020,7.701
4,Algeria,Wind,2016,19.400
8,Argentina,Wind,2020,9412.333
12,Argentina,Wind,2016,547.000
33,Armenia,Wind,2020,1.869
...,...,...,...,...
2207,Vanuatu,Wind,2016,5.416
2216,Venezuela (Bolivar. Rep.),Wind,2020,88.000
2220,Venezuela (Bolivar. Rep.),Wind,2016,88.000
2225,Viet Nam,Wind,2020,976.000


## Merge Solar and Wind

In [418]:
# merge solar and wind trimmed df's, inner, on country and year
combined_df = solar_df_trim.merge(wind_df_trim, how='inner', on=["Country or Area","Year"])
combined_df

,Country or Area,Technology_x,Year,"Quantity (kWh, million)_x",Technology_y,"Quantity (kWh, million)_y"
0,Algeria,Solar,2020,665.220,Wind,7.701
1,Algeria,Solar,2016,339.100,Wind,19.400
2,Argentina,Solar,2020,1344.589,Wind,9412.333
3,Argentina,Solar,2016,14.000,Wind,547.000
4,Armenia,Solar,2020,136.577,Wind,1.869
...,...,...,...,...,...,...
233,Vanuatu,Solar,2020,7.130,Wind,5.620
234,Vanuatu,Solar,2016,2.740,Wind,5.416
235,Venezuela (Bolivar. Rep.),Solar,2020,8.630,Wind,88.000
236,Venezuela (Bolivar. Rep.),Solar,2016,7.000,Wind,88.000


In [419]:
# rename columns(Quantity_x = Solar Quantity (kWh, Million), Quantity_y = Wind Quantity (kWh, Million) )
combined_df.rename(columns = {'Quantity (kWh, million)_x': 'Solar Quantity (kWh, Million)',
                             'Quantity (kWh, million)_y': 'Wind Quantity (kWh, Million)'}, inplace = True)

# rename countries to match investment df
combined_df['Country or Area'] = combined_df['Country or Area'].replace({'Bolivia (Plur. State of)'
                                                                                   : 'Bolivia'})

# drop columns technology_x and technology_y
combined_df = combined_df.drop(columns=["Technology_x","Technology_y"])

combined_df

,Country or Area,Year,"Solar Quantity (kWh, Million)","Wind Quantity (kWh, Million)"
0,Algeria,2020,665.220,7.701
1,Algeria,2016,339.100,19.400
2,Argentina,2020,1344.589,9412.333
3,Argentina,2016,14.000,547.000
4,Armenia,2020,136.577,1.869
...,...,...,...,...
233,Vanuatu,2020,7.130,5.620
234,Vanuatu,2016,2.740,5.416
235,Venezuela (Bolivar. Rep.),2020,8.630,88.000
236,Venezuela (Bolivar. Rep.),2016,7.000,88.000


In [420]:
# write to csv for use in summary workbook
combined_df.to_csv("resources/combined_solar_wind.csv", index=False)


## Investment Data

In [421]:
# read investment file and create dataframe
# spaces after delimiter in file so need to skip
investment_df = pd.read_csv("resources/renewable_investment.csv",skipinitialspace=True)

investment_df.head()

,ISO-code,Country/Area,Region,Project,Donor,Agency,Year,Category,Technology,Sub-technology,Finance Group,Finance Type,Source,Reference Date,Amount (2020 USD million)
0,EUE,European Union (27),Europe,SOLAS SUSTAINABLE ENERGY FUND,EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,https://www.eib.org/en/projects/loans/all/2018...,12/17/2020,0.583190
1,EUE,European Union (27),Europe,Taaleri Energia Solarwind Fund II,EU Institutions,European Investment Bank,2019,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,https://www.eib.org/en/projects/loans/all/2018...,6/6/2019,4.882124
2,EUE,European Union (27),Europe,MERIDIAM GREEN IMPACT GROWTH FUND (GIGF),EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,https://www.eib.org/en/projects/loans/all/2019...,12/7/2020,0.680388
3,EUE,European Union (27),Europe,SMALL-SCALE SOLAR PROGRAM IN SSA,EU Institutions,European Investment Bank,2020,Renewables,Solar energy,Off-grid Solar photovoltaic,Debt,Standard loan,https://www.eib.org/en/projects/loans/all/2019...,12/4/2020,0.777586
4,EUE,European Union (27),Europe,ITALIAN ENERGY EFFICIENCY FUND II,EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,https://www.eib.org/en/projects/loans/all/2019...,8/6/2020,0.155517


In [422]:
# change Year column from dtype int to dtype string (object)
investment_df = investment_df.astype({"Year": str}, errors='raise')

# check to see dtype changed
investment_df.dtypes

ISO-code                      object
Country/Area                  object
Region                        object
Project                       object
Donor                         object
Agency                        object
Year                          object
Category                      object
Technology                    object
Sub-technology                object
Finance Group                 object
Finance Type                  object
Source                        object
Reference Date                object
Amount (2020 USD million)    float64
dtype: object

In [423]:
# remove Reference Date, Source, Project columns 
investment_df = investment_df.drop(columns=["Project","Source","Reference Date"])

# check to see all rows are complete
investment_df.count()

ISO-code                     16524
Country/Area                 16524
Region                       16524
Donor                        16524
Agency                       16524
Year                         16524
Category                     16524
Technology                   16524
Sub-technology               16524
Finance Group                16524
Finance Type                 16524
Amount (2020 USD million)    16524
dtype: int64

In [424]:
# CLEANING (investment)

# rename country/area to country or area 
investment_df.rename(columns = {"Country/Area": "Country or Area"}, inplace=True)

# trim down to only rows with Year = 2016, 2017, 2018, 2019, 2020
investment_df = investment_df.loc[(investment_df["Year"] == "2020") |
                                (investment_df["Year"] == "2019") |
                                (investment_df["Year"] == "2018") |
                                (investment_df["Year"] == "2017") |
                                (investment_df["Year"] == "2016")] 

# trim down to rows with Category = Renewables
investment_df = investment_df.loc[investment_1["Category"] == "Renewables"]

investment_df

# FILTERING BY WIND/SOLAR ONLY = NOT ENOUGH DATA. BELOW IS WORK THAT IS NOW COMMENTED OUT
# USING ALL RENEWABLE INVESTMENT INSTEAD

# trim down to rows with Technology = Solar energy, Wind energy
#investment_df = investment_df.loc[(investment_df["Technology"] == "Solar energy") |
                               #(investment_df["Technology"] == "Wind energy")]

# change values in Technology column to "Solar" and "Wind"
#investment_df['Technology'] = investment_df['Technology'].replace({'Solar energy': 'Solar'})
#investment_df['Technology'] = investment_df['Technology'].replace({'Wind energy': 'Wind'})

,ISO-code,Country or Area,Region,Donor,Agency,Year,Category,Technology,Sub-technology,Finance Group,Finance Type,Amount (2020 USD million)
0,EUE,European Union (27),Europe,EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,0.583190
1,EUE,European Union (27),Europe,EU Institutions,European Investment Bank,2019,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,4.882124
2,EUE,European Union (27),Europe,EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,0.680388
3,EUE,European Union (27),Europe,EU Institutions,European Investment Bank,2020,Renewables,Solar energy,Off-grid Solar photovoltaic,Debt,Standard loan,0.777586
4,EUE,European Union (27),Europe,EU Institutions,European Investment Bank,2020,Renewables,Multiple renewables,Multiple renewables,Equity,Shares in collective investment vehicles,0.155517
...,...,...,...,...,...,...,...,...,...,...,...,...
16491,GBR,United Kingdom of Great Britain and Northern I...,Europe,United Kingdom,Green Investment Group,2016,Renewables,Bioenergy,Biogas,Debt,Standard loan,8.967915
16509,GBR,United Kingdom of Great Britain and Northern I...,Europe,United Kingdom,Green Investment Group,2017,Renewables,Wind energy,Offshore wind energy,Equity,Common equity,907.850627
16512,GBR,United Kingdom of Great Britain and Northern I...,Europe,Japan,Japan Bank for International Co-operation,2018,Renewables,Wind energy,Offshore wind energy,Debt,Standard loan,1010.455056
16519,GBR,United Kingdom of Great Britain and Northern I...,Europe,EU Institutions,European Investment Bank,2018,Renewables,Bioenergy,Renewable municipal waste,Debt,Standard loan,10.192514


In [425]:
# write to csv for backup
investment_df.to_csv("resources/investment_backup.csv", index=False)

## Trimming & Merging Investment Data

In [426]:
# trim investment data down to years 2020 and 2016
invest_trim = investment_df.loc[(investment_df["Year"] == "2020") | 
                                (investment_df["Year"] == "2016")]

# groupby and new df for analysis in comparison to combined_df
invest_final = pd.DataFrame(invest_trim.groupby(["Country or Area","Year"]).sum()["Amount (2020 USD million)"])

#reset index as columns
invest_final.reset_index(["Country or Area","Year"], inplace=True)

# rename Amount column to Investment (USD Million)
invest_final.rename(columns = {"Amount (2020 USD million)": "Investment (USD Million)"}, inplace=True)

# fix some country names to parallel combined_df
invest_final['Country or Area'] = invest_final['Country or Area'].replace({'Bolivia (Plurinational State of)'
                                                                                   : 'Bolivia','Kosovo*':'Kosovo',
                                                                                  'Republic of North Macedonia':'North Macedonia',
                                                                                  'United Kingdom of Great Britain and Northern Ireland':
                                                                                  'United Kingdom'})
invest_final

,Country or Area,Year,Investment (USD Million)
0,Afghanistan,2016,0.862826
1,Afghanistan,2020,18.575668
2,Algeria,2016,0.158601
3,Algeria,2020,0.000801
4,Angola,2016,93.957505
...,...,...,...
276,Yemen,2020,1.048830
277,Zambia,2016,109.513739
278,Zambia,2020,12.340080
279,Zimbabwe,2016,0.032496


In [427]:
# double check names of countries
invest_final["Country or Area"].unique()

array(['Afghanistan', 'Algeria', 'Angola', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Austria', 'Azerbaijan', 'Bahamas',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', 'Cuba', "Côte d'Ivoire",
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Eswatini', 'Ethiopia', 'European Union (27)', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Ghana', 'Greece',
       'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti',
       'Honduras', 'Iceland', 'India', 'Indonesia',
       'Iran (Islamic Republic of)',

In [428]:
# double check names of countries
combined_df["Country or Area"].unique()

array(['Algeria', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria',
       'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium',
       'Bhutan', 'Bolivia', 'Bonaire, St Eustatius, Saba',
       'Bosnia and Herzegovina', 'Brazil', 'British Virgin Islands',
       'Bulgaria', 'Cabo Verde', 'Canada', 'Chile', 'China', 'Colombia',
       'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia',
       'Denmark', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Eritrea', 'Estonia', 'Ethiopia', 'Faeroe Islands',
       'Falkland Is. (Malvinas)', 'Fiji', 'Finland', 'France',
       'French Polynesia', 'Gambia', 'Germany', 'Greece', 'Guam',
       'Guatemala', 'Guyana', 'Honduras', 'Hungary', 'India', 'Indonesia',
       'Iran (Islamic Rep. of)', 'Ireland', 'Israel', 'Italy', 'Jamaica',
       'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, Republic of',
       'Kosovo', 'Kuwait', 'Latvia', 'Lithuania', 'Luxembourg',
       'Madagascar', 'Mal

In [429]:
# write to csv for use in summary workbook
invest_final.to_csv("resources/investment_totals.csv", index=False)

## Test Merge of Wind/Solar df w/ Investment df

102 null values in investment column so decided to keep datasets seperate for comparison so as to not lose too much data.

In [430]:
test_merge = combined_df.merge(invest_final, how='left', on=["Country or Area","Year"])
test_merge

,Country or Area,Year,"Solar Quantity (kWh, Million)","Wind Quantity (kWh, Million)",Investment (USD Million)
0,Algeria,2020,665.220,7.701,0.000801
1,Algeria,2016,339.100,19.400,0.158601
2,Argentina,2020,1344.589,9412.333,5.810334
3,Argentina,2016,14.000,547.000,250.164444
4,Armenia,2020,136.577,1.869,24.716553
...,...,...,...,...,...
233,Vanuatu,2020,7.130,5.620,0.110954
234,Vanuatu,2016,2.740,5.416,NaN
235,Venezuela (Bolivar. Rep.),2020,8.630,88.000,NaN
236,Venezuela (Bolivar. Rep.),2016,7.000,88.000,NaN


In [431]:
#df['column name'].isna().sum()
test_merge["Investment (USD Million)"].isna().sum()

102